In [ ]:
#!/usr/bin/env uv run

# /// script
# requires-python = ">=3.10"
# dependencies = [
#     "crewai",
#     "crewai-tools",
#     "python-dotenv",
#     "youtube-transcript-api",
# ]
# ///

In [37]:
from youtube_transcript_api import YouTubeTranscriptApi
from crewai.tools import BaseTool
from crewai import Agent, LLM, Task, Crew, Process
import os
import re
from dotenv import load_dotenv

load_dotenv()

True

In [38]:
gemini_api_key = os.getenv('GEMINI_API_KEY')

# Use Gemini 2.5 Pro Experimental model
gemini_llm = LLM(
    model='gemini/gemini-2.0-flash',
    api_key=gemini_api_key,
    provider='google',
    temperature=0.1  # Lower temperature for more consistent results.
)

In [39]:
class fetchYoutubeTranscript(BaseTool):
    name: str = "Fetch YouTube Transcript"
    description: str = "Fetches the transcript of a youtube video using the provided link"

    def _run(self, link: str) -> str:
        video_id = re.search(r'(?:v=|/shorts/|youtu\.be/)([A-Za-z0-9_-]{6,})', link)
        if not video_id:
            raise ValueError("Could not parse YouTube video id from URL")
        video_id = video_id.group(1)

        ytt_api = YouTubeTranscriptApi()
        transcript_raw = ytt_api.fetch(video_id)

        transcript_clean = ""

        for snippet in transcript_raw:
            transcript_clean += snippet.text.replace("\n", " ") + " "

        return transcript_clean

In [40]:
fetch_youtube_transcript = fetchYoutubeTranscript()

In [41]:
# 1. Content Planner
content_planner = Agent(
    role="Content Planner",
    goal="Create a clear outline of the YouTube video transcript with relevant headings and subheadings.",
    backstory=(
        "You are an expert content strategist who specializes in extracting "
        "the structure and key talking points from transcripts to make them "
        "easy to digest as outlines."
    ),
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm
)

# 2. Video Summarizer
video_summarizer = Agent(
    role="YouTube Video Summarizer",
    goal="Summarize the YouTube transcript into a clear, structured summary with required headings and subheadings.",
    backstory=(
        "You specialize in condensing long YouTube transcripts into readable "
        "summaries. You emphasize clarity, brevity, and proper formatting."
    ),
    verbose=True,
    allow_delegation=False,
    tools=[fetch_youtube_transcript],
    llm=gemini_llm  # <-- your configured Gemini LLM
)

# 3. Editor (Fact Checker)
editor = Agent(
    role="Editor & Fact Checker",
    goal=(
        "Proofread and fact-check the final summary against the transcript. "
        "Ensure grammar, spelling, and structure are correct, and validate "
        "that all key points in the transcript are accurately represented."
    ),
    backstory=(
        "You are a meticulous editor with a keen eye for detail. "
        "You check not only for readability but also for accuracy, ensuring "
        "the summary faithfully reflects the transcript."
    ),
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm
)

In [ ]:
# 1. Fetch Transcript
fetch_transcript_task = Task(
    description=(
        "Fetch the transcript for the provided YouTube video link: {youtube_link}. "
        "Return only the raw transcript text."
    ),
    expected_output="The complete transcript text of the YouTube video.",
    agent=video_summarizer,
    tools=[fetch_youtube_transcript],
)

# 2. Content Outline
content_planner_task = Task(
    description=(
        "Using the transcript, create an outline of the video. "
        "The outline must contain clear headings and subheadings "
        "that represent the main topics and sections of the transcript."
    ),
    expected_output="An organized outline with relevant headings and subheadings.",
    agent=content_planner,
)

# 3. Summarization
summarization_task = Task(
    description=(
        "Using the transcript, write a structured summary of the YouTube video. "
        "Include headings and subheadings as appropriate, and keep it concise "
        "while preserving key details. Make sure to markdown the output"
    ),
    expected_output="A clear, structured summary of the transcript.",
    agent=video_summarizer,
)

# 4. Editing & Fact Checking
editing_task = Task(
    description=(
        "Proofread and fact-check the structured summary against the transcript. "
        "Ensure grammar, spelling, and readability are excellent. "
        "Verify that all important points in the transcript are covered accurately. "
        "Your final answer MUST be the polished, fact-checked summary."
    ),
    expected_output="A polished and fact-checked final summary.",
    agent=editor,
)

In [43]:
crew = Crew(
    agents=[content_planner, video_summarizer, editor],
    tasks=[fetch_transcript_task, content_planner_task, summarization_task, editing_task],
    process=Process.sequential,
)

In [44]:
result = crew.kickoff(inputs={'youtube_link': "https://www.youtube.com/watch?v=XB4MIexjvY0"})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: YouTube Video Summarizer                                                                                │
│                                                                                                                 │
│  Task: Fetch the transcript for the provided YouTube video link: https://www.youtube.com/watch?v=XB4MIexjvY0.   │
│  Return only the raw transcript text.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: YouTube Video Summarizer                                                                                │
│                                                                                                                 │
│  Thought: I need to fetch the transcript of the YouTube video using the provided link.                          │
│                                                                                                                 │
│  Using Tool: Fetch YouTube Transcript                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"link\": \"https://www.youtube.com/watch?v=XB4MIexjvY0\"}"                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  topic is dtra algorithm this algorithm is for single Source shortest path problem if a weighted graph is       │
│  given then we have to find our shortest path from some starting vertex to all of the vertexes let us say I'm   │
│  selecting starting vertex as one then I have to find out shortest path to all the vertexes maybe a direct      │
│  path or via other vertices and I can select any one of the vertex as a source vertex as we have to find out a  │
│  shortest path so it's a minimization problem and the minimization problem is an optimization problem so        │
│  optimization problems can be solved using greedy method greedy method says that a problem should be solved in  │
│  stages by taking one step at a time and considering one input at a time to get a optimal solution and in       │
│  Greedy method there are predefined procedures and we follow that procedure to get a optimal solution so Dix    │
│  algorithm gives a procedure for getting a optimal solution that is minimum result that is shortest part in     │
│  this video I will show how a dixar algorithms works and the disra algorithm can work on a directed as well as  │
│  non-directed graph and also I will show you the drawback of dixa algorithm let us see the approach of dixar    │
│  algorithm for that I'll show you the base basic thing that is followed by dexra so for that first of all I'll  │
│  take a very small example and show you to show the approach of Dix algorithm I have taken a very small graph   │
│  here from this we will understand what is the approach of thra if I say one is the starting vertex and I want  │
│  to find the shortest path to two as well as three so if you see from this graph there is a direct path to Vex  │
│  2 so its cost is 2 and there is no direct path to three so we don't know what is the path first of all we      │
│  consider a direct Edge path but now if we follow this algorithm we'll be selecting first the shortest path     │
│  vertex so this is shortest because this is two that is infinity so we'll select this one and Di says that      │
│  once you have selected one of the shortest path then chec...                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: YouTube Video Summarizer                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  topic is dtra algorithm this algorithm is for single Source shortest path problem if a weighted graph is       │
│  given then we have to find our shortest path from some starting vertex to all of the vertexes let us say I'm   │
│  selecting starting vertex as one then I have to find out shortest path to all the vertexes maybe a direct      │
│  path or via other vertices and I can select any one of the vertex as a source vertex as we have to find out a  │
│  shortest path so it's a minimization problem and the minimization problem is an optimization problem so        │
│  optimization problems can be solved using greedy method greedy method says that a problem should be solved in  │
│  stages by taking one step at a time and considering one input at a time to get a optimal solution and in       │
│  Greedy method there are predefined procedures and we follow that procedure to get a optimal solution so Dix    │
│  algorithm gives a procedure for getting a optimal solution that is minimum result that is shortest part in     │
│  this video I will show how a dixar algorithms works and the disra algorithm can work on a directed as well as  │
│  non-directed graph and also I will show you the drawback of dixa algorithm let us see the approach of dixar    │
│  algorithm for that I'll show you the base basic thing that is followed by dexra so for that first of all I'll  │
│  take a very small example and show you to show the approach of Dix algorithm I have taken a very small graph   │
│  here from this we will understand what is the approach of thra if I say one is the starting vertex and I want  │
│  to find the shortest path to two as well as three so if you see from this graph there is a direct path to Vex  │
│  2 so its cost is 2 and there is no direct path to three so we don't know what is the path first of all we      │
│  consider a direct Edge path but now if we follow this algorithm we'll be selecting first the shortest path     │
│  vertex so this is shortest because this is two that is infinity so we'll select this one and Di says that      │
│  once you have selected one of the shortest path then check via that vertex from that vertex there is any       │
│  shortest path to other vertices so let us check who is connected to two so from two we can go three this is 2  │
│  + 4 that is 6 and right now it is infinity yes this Infinity can be changed to six this means that there is a  │
│  shortest path from verx 1 to 3 of distance 6 no doubt it's not a direct path it's coming via two but there is  │
│  a path that's how the Dix algorithm always select a Vertex with a short shortest path then it will update the  │
│  shortest path to other vertices if possible and this updation is called as relaxation let me show you how      │
│  this relaxation works for that I will call this vertex as U and this vertex as V and this is distance of U     │
│  and this is distance of V initially it was Infinity now it is change to six and this is cost of an H from U    │
│  to V so here relaxation means if the distance of vertex U + cost of an edge U comma V is less than distance    │
│  of vertex V see here the distance of U was 2 and the cost of an edge was 4 and this was initially Infinity so  │
│  this 2 + 4 is less than infinity yes so we have modified this to six so we'll modify this one that is          │
│  distance of V is modified to distance of U Plus cost o

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"link\": \"https://www.youtube.com/watch?v=pVfj6mxhdfw\"}"                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error:                                       │
│  Could not retrieve a transcript for the video https://www.youtube.com/watch?v=pVfj6mxhdfw! This is most        │
│  likely caused by:                                                                                              │
│                                                                                                                 │
│  The video is no longer available                                                                               │
│                                                                                                                 │
│  If you are sure that the described cause is not responsible for this error and that a transcript should be     │
│  retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add    │
│  which version of youtube_transcript_api you are using and provide the information needed to replicate the      │
│  error. Also make sure that there are no open issues which already describe your problem!.                      │
│   Tool Fetch YouTube Transcript accepts these inputs: Tool Name: Fetch YouTube Transcript                       │
│  Tool Arguments: {'link': {'description': None, 'type': 'str'}}                                                 │
│  Tool Description: Fetches the transcript of a youtube video using the provided link.                           │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [Fetch YouTube Transcript]                                        │
│  Action Input: the input to the action, dictionary enclosed in curly braces                                     │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│  This Thought/Action/Action Input/Result can repeat N times. Once I know the final answer, I must return the    │
│  following format:                                                                                              │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now can give a great answer                                                                         │
│  Final Answer: Your final answer must be the great and the most complete as possible, it must be outcome        │
│  described                                                                                                      │
│                                                                                                                 │
│  ```                                                                                                            │
│                                                       

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: YouTube Video Summarizer                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  I. Introduction to Dijkstra's Algorithm                                                                        │
│      A. Definition: Single Source Shortest Path Problem                                                         │
│          1. Finding the shortest path from a starting vertex to all other vertices in a weighted graph.         │
│          2. Source vertex selection.                                                                            │
│      B. Dijkstra's Algorithm as a Greedy Method                                                                 │
│          1. Optimization problem: Minimization of path length.                                                  │
│          2. Greedy approach: Solving in stages, one step at a time.                                             │
│          3. Predefined procedures for optimal solution.                                                         │
│      C. Overview of the Video                                                                                   │
│          1. How Dijkstra's algorithm works.                                                                     │
│          2. Applicability to directed and non-directed graphs.                                                  │
│          3. Drawbacks of Dijkstra's algorithm.                                                                  │
│                                                                                                                 │
│  II. Approach of Dijkstra's Algorithm: A Simple Example                                                         │
│      A. Illustrative Graph                                                                                      │
│          1. Finding the shortest path from vertex 1 to vertices 2 and 3.                                        │
│          2. Direct path vs. path via other vertices.                                                            │
│      B. Core Idea: Selecting the Shortest Path Vertex                                                           │
│          1. Initial selection of the shortest path vertex.                                                      │
│          2. Checking for shorter paths to other vertices via the selected vertex.                               │
│      C. Relaxation: Updating Shortest Paths                                                                     │
│          1. Definition of relaxation: Updating distances if a shorter path is found.                            │
│          2. Formula for relaxation:                                                                             │
│              a. If distance(U) + cost(U, V) < distance(V), then update distance(V).                             │
│      D. Applying Relaxation                                                                                     │
│          1. Modifying the distance to vertex 3 via vertex 2.                                                    │
│                                                                                                                 │
│  III. Solving a Graph Problem Using Dijkstra's Algorithm                                                        │
│      A. Initial Setup                                                                                           │
│          1. Selecting vertex 1 as the source vertex.   

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor & Fact Checker                                                                                   │
│                                                                                                                 │
│  Task: Proofread and fact-check the structured summary against the transcript. Ensure grammar, spelling, and    │
│  readability are excellent. Verify that all important points in the transcript are covered accurately. Your     │
│  final answer MUST be the polished, fact-checked summary.                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor & Fact Checker                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  I. Introduction to Dijkstra's Algorithm                                                                        │
│      A. Definition: Single Source Shortest Path Problem                                                         │
│          1. Finding the shortest path from a starting vertex to all other vertices in a weighted graph.         │
│          2. Source vertex selection.                                                                            │
│      B. Dijkstra's Algorithm as a Greedy Method                                                                 │
│          1. Optimization problem: Minimization of path length.                                                  │
│          2. Greedy approach: Solving in stages, one step at a time.                                             │
│          3. Predefined procedures for optimal solution.                                                         │
│      C. Overview of the Video                                                                                   │
│          1. How Dijkstra's algorithm works.                                                                     │
│          2. Applicability to directed and non-directed graphs.                                                  │
│          3. Drawbacks of Dijkstra's algorithm.                                                                  │
│                                                                                                                 │
│  II. Approach of Dijkstra's Algorithm: A Simple Example                                                         │
│      A. Illustrative Graph                                                                                      │
│          1. Finding the shortest path from vertex 1 to vertices 2 and 3.                                        │
│          2. Direct path vs. path via other vertices.                                                            │
│      B. Core Idea: Selecting the Shortest Path Vertex                                                           │
│          1. Initial selection of the shortest path vertex.                                                      │
│          2. Checking for shorter paths to other vertices via the selected vertex.                               │
│      C. Relaxation: Updating Shortest Paths                                                                     │
│          1. Definition of relaxation: Updating distances if a shorter path is found.                            │
│          2. Formula for relaxation:                                                                             │
│              a. If distance(U) + cost(U, V) < distance(V), then update distance(V).                             │
│      D. Applying Relaxation                                                                                     │
│          1. Modifying the distance to vertex 3 via vertex 2.                                                    │
│                                                                                                                 │
│  III. Solving a Graph Problem Using Dijkstra's Algorithm                                                        │
│      A. Initial Setup                                                                                           │
│          1. Selecting vertex 1 as the source vertex.   

In [47]:
with open("youtube_summary.md", "w", encoding="utf-8") as f:
    f.write(result.raw)

In [46]:
display(Markdown(result.raw))

I. Introduction to Dijkstra's Algorithm
    A. Definition: Single Source Shortest Path Problem
        1. Finding the shortest path from a starting vertex to all other vertices in a weighted graph.
        2. Source vertex selection.
    B. Dijkstra's Algorithm as a Greedy Method
        1. Optimization problem: Minimization of path length.
        2. Greedy approach: Solving in stages, one step at a time.
        3. Predefined procedures for optimal solution.
    C. Overview of the Video
        1. How Dijkstra's algorithm works.
        2. Applicability to directed and non-directed graphs.
        3. Drawbacks of Dijkstra's algorithm.

II. Approach of Dijkstra's Algorithm: A Simple Example
    A. Illustrative Graph
        1. Finding the shortest path from vertex 1 to vertices 2 and 3.
        2. Direct path vs. path via other vertices.
    B. Core Idea: Selecting the Shortest Path Vertex
        1. Initial selection of the shortest path vertex.
        2. Checking for shorter paths to other vertices via the selected vertex.
    C. Relaxation: Updating Shortest Paths
        1. Definition of relaxation: Updating distances if a shorter path is found.
        2. Formula for relaxation:
            a. If distance(U) + cost(U, V) < distance(V), then update distance(V).
    D. Applying Relaxation
        1. Modifying the distance to vertex 3 via vertex 2.

III. Solving a Graph Problem Using Dijkstra's Algorithm
    A. Initial Setup
        1. Selecting vertex 1 as the source vertex.
        2. Assigning initial distances to all vertices (0 for source, infinity for others without direct edges).
    B. Repeating Steps
        1. Step 1: Selecting the shortest path (vertex 2).
            a. Performing relaxation: Updating distances to connected vertices (3 and 4).
        2. Step 2: Selecting the next shortest path (vertex 3).
            a. Performing relaxation: Updating distances to connected vertices (5).
        3. Step 3: Selecting the next shortest path (vertex 5).
            a. Performing relaxation: Updating distances to connected vertices (6).
        4. Step 4: Selecting the next shortest path (vertex 4).
            a. Performing relaxation: Updating distances to connected vertices (6).
        5. Step 5: Selecting the last remaining vertex (vertex 6).
    C. Final Result
        1. Shortest path distances from vertex 1 to all other vertices.
        2. Listing the vertices and their corresponding shortest path distances.

IV. Time Complexity Analysis
    A. Finding Shortest Paths to All Vertices
        1. N vertices in total.
    B. Relaxation Process
        1. Relaxing connected vertices.
        2. Worst-case scenario: Relaxing all N vertices from each vertex.
    C. Time Complexity
        1. O(N*N) or O(N^2) in the worst case (complete graph).
        2. Theta(V^2) or Theta(N^2)

V. Dijkstra's Algorithm on a Weighted Directed Graph
    A. Table Representation
        1. Using a table to represent and update distances.
        2. Initial distances based on direct edges from the source vertex.
    B. Step-by-Step Solution
        1. Selecting vertex 4 (shortest distance).
            a. Relaxing vertex 5.
        2. Selecting vertex 5.
            a. Relaxing vertices 2 and 3.
        3. Selecting vertex 2.
            a. Attempting to relax vertex 3 (no change).
        4. Selecting vertex 3.
            a. Attempting to relax vertex 5 (no change).
        5. Selecting vertex 6 (remaining infinity).
    C. Final Result
        1. Shortest path distances to all reachable vertices.
        2. Vertex 6 remains unreachable (infinity).

VI. Dijkstra's Algorithm on Non-Directed Graphs
    A. Conversion to Directed Graph
        1. Replacing non-directed edges with parallel directed edges.
    B. Applicability
        1. Dijkstra's algorithm works on both directed and non-directed graphs.

VII. Drawback of Dijkstra's Algorithm: Negative Weighted Edges
    A. Introduction to Negative Edges
        1. Edges represent values, not necessarily distances.
        2. Real-world problems can have negative values.
    B. Example 1: Negative Edge (-3)
        1. Dijkstra's algorithm works correctly.
    C. Example 2: Negative Edge (-10)
         1. Dijkstra's algorithm fails to find the correct shortest path.
         2. The algorithm's greediness leads to a suboptimal solution.
         3. The algorithm does not revisit relaxed vertices, missing a better path.

VIII. Conclusion
    A. Dijkstra's Algorithm and Greedy Approach
        1. The algorithm's greediness can fail with negative edges.
    B. Alternative Solution: Bellman-Ford Algorithm
        1. Mention of Bellman-Ford algorithm for handling negative edges (dynamic programming approach).
    C. Closing Remarks
        1. Summary of topics covered.
        2. Encouragement for comments and suggestions.

## Dijkstra's Algorithm Explained

### Introduction

Dijkstra's algorithm is a method for solving the single-source shortest path problem. Given a weighted graph, the algorithm finds the shortest path from a designated starting vertex to all other vertices in the graph. Since finding the shortest path is a minimization problem, Dijkstra's algorithm can be viewed as an optimization problem solvable by a greedy method. The greedy method solves the problem in stages, making one step at a time to achieve an optimal solution by following a predefined procedure.

### Algorithm Approach

The algorithm works on both directed and non-directed graphs. The basic approach involves initially considering direct paths from the starting vertex. The algorithm then iteratively selects the vertex with the shortest known path and updates the shortest paths to other vertices if a shorter path is found by going through the selected vertex. This process of updating the shortest paths is called relaxation.

**Relaxation:** If the distance to vertex U plus the cost of the edge from U to V is less than the current distance to V, then the distance to V is updated.

### Solving a Graph Problem

1.  **Initialization:** Assign initial distances to all vertices based on direct edges from the source vertex. If there is no direct edge, assign infinity.
2.  **Iteration:**
    *   Select the vertex with the shortest known distance.
    *   For each neighbor of the selected vertex, attempt to relax the distance to that neighbor.
    *   Repeat until all vertices have been selected.

### Time Complexity

The algorithm finds the shortest path to all *n* vertices. In the worst-case scenario (complete graph), each vertex might need to relax all other vertices, resulting in a time complexity of O(n^2).

### Weighted Directed Graph Example

The video provides an example using a weighted directed graph and demonstrates how to use a table to keep track of distances and selected vertices. The algorithm iteratively selects the vertex with the minimum distance and relaxes the distances to its neighbors, updating the table accordingly.

### Non-Directed Graphs

Dijkstra's algorithm can also be applied to non-directed graphs. Non-directed edges can be converted into directed edges by adding parallel edges in both directions.

### Drawbacks: Negative Weighted Edges

Dijkstra's algorithm does not work correctly when the graph contains negative weighted edges. The algorithm's greedy approach of always selecting the shortest path seen so far can lead to incorrect results because a negative edge could potentially reduce the path length of an already "relaxed" vertex.

**Example:** The video demonstrates a case where a negative edge leads to the algorithm finding a suboptimal path.

### Conclusion

Dijkstra's algorithm is a greedy algorithm that efficiently finds the shortest paths in graphs with non-negative edge weights. However, it fails to produce correct results in the presence of negative edge weights. For graphs with negative edges, the Bellman-Ford algorithm provides an alternative solution using dynamic programming.